## Model Training

#### 1.1 Import Data and Required Packages
##### Importing Pandas, Numpy, Matplotlib, Seaborn and Warings Library.

In [13]:
# Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import  StandardScaler
from category_encoders import OneHotEncoder

import warnings

warnings.filterwarnings('ignore')

In [4]:
df_train=pd.read_csv('data/train_F3fUq2S.csv')
df_test=pd.read_csv('data/test_Bk2wfZ3.csv')

In [5]:
df_train.head(5)

,campaign_id,sender,subject_len,body_len,mean_paragraph_len,day_of_week,is_weekend,times_of_day,category,product,...,is_image,is_personalised,is_quote,is_timer,is_emoticons,is_discount,is_price,is_urgency,target_audience,click_rate
0,1,3,76,10439,39,5,1,Noon,6,26,...,0,0,0,0,0,0,0,0,14,0.103079
1,2,3,54,2570,256,5,1,Morning,2,11,...,0,0,0,0,0,0,0,0,10,0.700000
2,3,3,59,12801,16,5,1,Noon,2,11,...,1,0,1,0,0,0,0,0,16,0.002769
3,4,3,74,11037,30,4,0,Evening,15,9,...,0,0,0,0,0,0,0,0,10,0.010868
4,5,3,80,10011,27,5,1,Noon,6,26,...,0,0,1,0,0,0,0,0,14,0.142826


In [163]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1888 entries, 0 to 1887
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   campaign_id         1888 non-null   int64  
 1   sender              1888 non-null   int64  
 2   subject_len         1888 non-null   int64  
 3   body_len            1888 non-null   int64  
 4   mean_paragraph_len  1888 non-null   int64  
 5   day_of_week         1888 non-null   int64  
 6   is_weekend          1888 non-null   int64  
 7   times_of_day        1888 non-null   object 
 8   category            1888 non-null   int64  
 9   product             1888 non-null   int64  
 10  no_of_CTA           1888 non-null   int64  
 11  mean_CTA_len        1888 non-null   int64  
 12  is_image            1888 non-null   int64  
 13  is_personalised     1888 non-null   int64  
 14  is_quote            1888 non-null   int64  
 15  is_timer            1888 non-null   int64  
 16  is_emo

In [7]:
df_train

,campaign_id,sender,subject_len,body_len,mean_paragraph_len,day_of_week,is_weekend,times_of_day,category,product,...,is_image,is_personalised,is_quote,is_timer,is_emoticons,is_discount,is_price,is_urgency,target_audience,click_rate
0,1,3,76,10439,39,5,1,Noon,6,26,...,0,0,0,0,0,0,0,0,14,0.103079
1,2,3,54,2570,256,5,1,Morning,2,11,...,0,0,0,0,0,0,0,0,10,0.700000
2,3,3,59,12801,16,5,1,Noon,2,11,...,1,0,1,0,0,0,0,0,16,0.002769
3,4,3,74,11037,30,4,0,Evening,15,9,...,0,0,0,0,0,0,0,0,10,0.010868
4,5,3,80,10011,27,5,1,Noon,6,26,...,0,0,1,0,0,0,0,0,14,0.142826
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1883,1884,3,88,1451,75,2,0,Noon,2,11,...,0,0,1,0,0,0,0,0,10,0.350746
1884,1885,3,58,10537,40,2,0,Evening,2,11,...,0,0,0,0,0,0,0,0,11,0.004728
1885,1886,3,89,11050,26,1,0,Evening,15,9,...,0,0,0,0,0,0,0,0,6,0.008289
1886,1887,3,58,10537,40,1,0,Evening,2,11,...,0,0,0,0,0,0,0,0,16,0.012014


### Data Preprocessing

From the  data analysis made we came to know that some features are dominating in train dataset.
Also if we can check statistically those features having very correlation with target features.
Those features can ve dropeed from train and test data set.

In [8]:
def handle_outliers(df,feature):

    IQR=(df[feature].quantile(0.75))-(df[feature].quantile(0.25))
    lower_limit=df[feature].quantile(0.25)-(IQR * 1.5)
    upper_limit=df[feature].quantile(0.75)+(IQR * 1.5)

    df_train.loc[df[feature]>=upper_limit,feature]=upper_limit
    df_train.loc[df[feature]<=lower_limit,feature]=lower_limit

    return df[feature]

In [9]:
outlier_features=['no_of_CTA','mean_paragraph_len','body_len','subject_len','mean_CTA_len']

for i in range(len(outlier_features)):
    handle_outliers(df_train,outlier_features[i])

In [14]:
def onehot_encoders(feature, train_data, test_data):
    encoder_ = OneHotEncoder(cols=[feature],handle_unknown='ignore')
    train_encoded = encoder_.fit_transform(train_data)
    print(train_encoded)
    # test_encoded = encoder_.transform(test_data)

    # Update the original DataFrames with the encoded values
    train_data.drop(columns=[feature], inplace=True)
    train_data[encoder_.get_feature_names_out([feature])] = train_encoded

# Apply one-hot encoding to both training and test sets
onehot_encoding_features = [ 'is_weekend', 'is_personalised', 'is_discount', 'is_price', 'is_urgency','day_of_week','times_of_day']

for feature in onehot_encoding_features:
    onehot_encoders(feature, df_train, df_test)

      campaign_id  sender  subject_len  body_len  mean_paragraph_len  \
0               1       3         76.0     10439                39.0   
1               2       3         54.0      2570                68.5   
2               3       3         59.0     12801                16.0   
3               4       3         74.0     11037                30.0   
4               5       3         80.0     10011                27.0   
...           ...     ...          ...       ...                 ...   
1883         1884       3         88.0      1451                68.5   
1884         1885       3         58.0     10537                40.0   
1885         1886       3         89.0     11050                26.0   
1886         1887       3         58.0     10537                40.0   
1887         1888       3         89.0     11050                26.0   

      day_of_week  is_weekend_1  is_weekend_2 times_of_day  category  ...  \
0               5             1             0         Noon

In [15]:
df_train.drop(columns=['campaign_id'],inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1888 entries, 0 to 1887
Data columns (total 41 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   campaign_id         1888 non-null   int64  
 1   sender              1888 non-null   int64  
 2   subject_len         1888 non-null   float64
 3   body_len            1888 non-null   int64  
 4   mean_paragraph_len  1888 non-null   float64
 5   category            1888 non-null   int64  
 6   product             1888 non-null   int64  
 7   no_of_CTA           1888 non-null   float64
 8   mean_CTA_len        1888 non-null   int64  
 9   is_image            1888 non-null   int64  
 10  is_quote            1888 non-null   int64  
 11  is_timer            1888 non-null   int64  
 12  is_emoticons        1888 non-null   int64  
 13  target_audience     1888 non-null   int64  
 14  click_rate          1888 non-null   float64
 15  is_weekend_1        1888 non-null   int64  
 16  is_wee

#### Model Training-:


In [176]:
X=df_train.drop(['click_rate'] ,axis=1)
Y=df_train['click_rate']

In [177]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid=train_test_split(X,Y,random_state=42,test_size=0.2)

In [155]:
print('Train', X_train.shape, y_train.shape)
print('Validation', X_valid.shape, y_valid.shape)

Train (1510, 14) (1510,)
Validation (378, 14) (378,)


In [178]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

X_train=scaler.fit_transform(X_train)

X_valid=scaler.transform(X_valid)

In [1]:
X_train

NameError: name 'X_train' is not defined

In [191]:
#### base model

mean_pred=np.repeat(y_train.mean(),len(y_valid))
r2_score(y_valid,mean_pred)

-0.012711622261606603

In [192]:
def evaluate_model(true,predicted):
    mae=mean_absolute_error(true,predicted)
    mse=mean_squared_error(true,predicted)
    r2score=r2_score(true,predicted)
    return mae,mse,r2score

In [193]:
model={
    "Linear Regresion":LinearRegression(),
    "Lasso":Lasso(),
    "Ridge":Ridge(),
    "KNeighborsRegressor":KNeighborsRegressor(),
    "DecisionTree":DecisionTreeRegressor(),
    "Random Forest":RandomForestRegressor(),
    "XGB Regressor":XGBRegressor(),
    "Catboot Rgressor":CatBoostRegressor(verbose=False),
    "AdaBoost Regressor":AdaBoostRegressor()
}

model_list=[]
r2_list=[]
for model_name,model in model.items():
    model.fit(X_train,y_train)

    y_train_pred=model.predict(X_train)
    y_valid_pred=model.predict(X_valid)

     # Evaluate Train and Test dataset
    model_train_mae, model_train_mse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae, model_test_mse, model_test_r2 = evaluate_model(y_valid, y_valid_pred)

    print(model_name)
    model_list.append(model_name)
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_mse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Validation set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_mse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('=' * 35)
    print('\n')

Linear Regresion
Model performance for Training set
- Root Mean Squared Error: 0.0064
- Mean Absolute Error: 0.0455
- R2 Score: 0.1757
----------------------------------
Model performance for Validation set
- Root Mean Squared Error: 0.0036
- Mean Absolute Error: 0.0419
- R2 Score: 0.1760


Lasso
Model performance for Training set
- Root Mean Squared Error: 0.0072
- Mean Absolute Error: 0.0483
- R2 Score: 0.0734
----------------------------------
Model performance for Validation set
- Root Mean Squared Error: 0.0039
- Mean Absolute Error: 0.0418
- R2 Score: 0.1120


Ridge
Model performance for Training set
- Root Mean Squared Error: 0.0064
- Mean Absolute Error: 0.0455
- R2 Score: 0.1757
----------------------------------
Model performance for Validation set
- Root Mean Squared Error: 0.0036
- Mean Absolute Error: 0.0419
- R2 Score: 0.1762


KNeighborsRegressor
Model performance for Training set
- Root Mean Squared Error: 0.0035
- Mean Absolute Error: 0.0304
- R2 Score: 0.5469
--------

In [196]:


# Split the data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, Y, test_size=0.2, random_state=42)

# Random Forest Model
# Define the Random Forest model
rf_model = RandomForestRegressor()

# Define the hyperparameter grid to search
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Use GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, scoring='r2')
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

# Train the model with the best hyperparameters
best_rf_model = RandomForestRegressor(**best_params)
best_rf_model.fit(X_train, y_train)

# Model performance on training set
train_predictions = best_rf_model.predict(X_train)
print("Model performance for Training set:")
print("- Root Mean Squared Error:", round(mean_squared_error(y_train, train_predictions, squared=False), 4))
print("- Mean Absolute Error:", round(mean_absolute_error(y_train, train_predictions), 4))
print("- R2 Score:", round(r2_score(y_train, train_predictions), 4))
print("----------------------------------")

# Model performance on validation set
valid_predictions = best_rf_model.predict(X_valid)
print("Model performance for Validation set:")
print("- Root Mean Squared Error:", round(mean_squared_error(y_valid, valid_predictions, squared=False), 4))
print("- Mean Absolute Error:", round(mean_absolute_error(y_valid, valid_predictions), 4))
print("- R2 Score:", round(r2_score(y_valid, valid_predictions), 4))
print("===================================")

# Feature Importance
feature_importance = pd.Series(best_rf_model.feature_importances_, index=X.columns)
feature_importance = feature_importance.sort_values(ascending=False)
print("Feature Importance:")
print(feature_importance)


Best Hyperparameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}
Model performance for Training set:
- Root Mean Squared Error: 0.0215
- Mean Absolute Error: 0.011
- R2 Score: 0.9402
----------------------------------
Model performance for Validation set:
- Root Mean Squared Error: 0.0497
- Mean Absolute Error: 0.0287
- R2 Score: 0.4417
Feature Importance:
body_len              3.033837e-01
campaign_id           1.339133e-01
no_of_CTA             1.005966e-01
mean_CTA_len          8.019620e-02
mean_paragraph_len    7.752161e-02
subject_len           7.700584e-02
category              4.065906e-02
product               3.139199e-02
is_quote              2.228750e-02
is_image              2.183809e-02
day_of_week_1         1.801682e-02
target_audience       1.522766e-02
times_of_day_3        8.186194e-03
is_personalised_2     7.818804e-03
day_of_week_5         7.368216e-03
day_of_week_3         7.236326e-03
sender                6.823491e-03
ti

In [194]:
#parameter grid for randomforest
params={
    'n_estimators':[100,150,200,250,300,350,400,450,500,750,1000],
    'criterion':['squared_error', 'absolute_error', 'poisson'],
    'oob_score':[True],
    'random_state':[101]
}

rf2=RandomForestRegressor()


random_search = RandomizedSearchCV(rf2, param_distributions=params,n_iter=5, n_jobs=-1, cv=5, scoring='r2')

random_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = random_search.best_params_
print("Best Hyperparameters:", best_params)

# Train the model with the best hyperparameters
best_rf_model = RandomForestRegressor(**best_params)
best_rf_model.fit(X_train, y_train)

# Model performance on training set
train_predictions = best_rf_model.predict(X_train)
print("Model performance for Training set:")
print("- Root Mean Squared Error:", round(mean_squared_error(y_train, train_predictions, squared=False), 4))
print("- Mean Absolute Error:", round(mean_absolute_error(y_train, train_predictions), 4))
print("- R2 Score:", round(r2_score(y_train, train_predictions), 4))
print("----------------------------------")

# Model performance on validation set
valid_predictions = best_rf_model.predict(X_valid)
print("Model performance for Validation set:")
print("- Root Mean Squared Error:", round(mean_squared_error(y_valid, valid_predictions, squared=False), 4))
print("- Mean Absolute Error:", round(mean_absolute_error(y_valid, valid_predictions), 4))
print("- R2 Score:", round(r2_score(y_valid, valid_predictions), 4))
print("===================================")

# Feature Importance
feature_importance = pd.Series(best_rf_model.feature_importances_, index=X.columns)
feature_importance = feature_importance.sort_values(ascending=False)
print("Feature Importance:")
print(feature_importance)



Best Hyperparameters: {'random_state': 101, 'oob_score': True, 'n_estimators': 400, 'criterion': 'poisson'}
Model performance for Training set:
- Root Mean Squared Error: 0.0215
- Mean Absolute Error: 0.0107
- R2 Score: 0.9405
----------------------------------
Model performance for Validation set:
- Root Mean Squared Error: 0.0448
- Mean Absolute Error: 0.0261
- R2 Score: 0.5465
Feature Importance:
body_len              2.185233e-01
no_of_CTA             1.541684e-01
campaign_id           1.509115e-01
mean_CTA_len          8.364541e-02
mean_paragraph_len    7.718052e-02
subject_len           7.412029e-02
category              6.790996e-02
target_audience       3.766440e-02
product               3.177187e-02
is_image              1.543001e-02
is_quote              1.370101e-02
times_of_day_3        8.832397e-03
day_of_week_1         7.538286e-03
sender                7.366617e-03
times_of_day_1        5.600617e-03
is_emoticons          5.427975e-03
day_of_week_3         5.039032e-03
da